# Excel2CSV converter to import data for SSBD-OMERO server

Excel files are converted into CSV files. <BR>
SH files are generated to import images and their metadata.<BR>
All files can be used to import images and their metadata into  SSBD-OMERO server (Ubuntu server).

Input data: 
- Metadata_template.xlsx
- project_name/project_name.xlsx

Output data (e.g.):
- metadata-all.csv (newly generated to check all generated metadata)<BR><BR>

- metadata/copyData.sh (newly generated to upload all data in all projects into SSBD-OMERO server)
- metadata/addAnnotations.sh (newly generated to import metadata of all images in all projects into SSBD-OMERO server)
- metadata/makeMD5.sh (new generate to make MD5 files for all images into SSBD~OMERO server) <BR><BR>

- metadata/project-name/project-name.xlsx (copy)
- metadata/project-name/project-name.csv (newly generated to grasp all metadata in one project)
- metadata/project-name/project-name_image_metadata.csv (newly generate to import metadata of images in one project into SSBD-OMERO server)
- metadata/project-name/project-name_image.csv (newly generaeete to import images in one project into SSBD-OMERO server)
- metadata/project-name/importImage.sh
- metadata/project-name/zipImage.sh

## Preparing library

In [221]:
# > Python 3
import pandas as pd
import os, sys, csv, re, glob, shutil

print (sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


##  Setting up the executing conditions

In [222]:
#INPUT_DIR = "/Volumes/datacollection/Collection_test/"
INPUT_DIR = "Collection_test/"
TEMPLATE_FILE = INPUT_DIR + 'Metadata_template.xlsx' # template file of metadata description

#OUTPUT_DIR  = "/Volumes/datacollection/Collection_test/metadata/"
OUTPUT_DIR  = "Collection_test/metadata/"

# SSBD-OMERO server information
SERVER_HOST  = "hostname" # hostname of ssbd-omero server
SERVER_USER  = "user" # username of ssbd-omero server 
SERVER_DIR = "/home/user/metadata/" # direcotry of ssbd-omero server

# Specify Project ID (PID)
TARGET_PROJECTS = [20, 115]
#TARGET_PROJECTS = [20]

print ("INPUT_DIR:", INPUT_DIR)
print ("OUTPUT_DIR:", OUTPUT_DIR)
print ("TEMPLATE_FILE:", TEMPLATE_FILE)
print ("TARGET_PROJECT:", TARGET_PROJECTS)
print ("SERVER_HOST:", SERVER_HOST)
print ("SERVER_USER:", SERVER_USER)
print ("SERVER_DIR:", SERVER_DIR)

if not os.path.isdir(INPUT_DIR):
    print ("ERROR: Input dir not accessed:" + INPUT_DIR)


INPUT_DIR: Collection_test/
OUTPUT_DIR: Collection_test/metadata/
TEMPLATE_FILE: Collection_test/Metadata_template.xlsx
TARGET_PROJECT: [20, 115]
SERVER_HOST: hostname
SERVER_USER: user
SERVER_DIR: /home/user/metadata/


## Loading functions

In [223]:
# Decoding scale and unit on X,Y,Z-axes
def searchXYZ (data):
    if pd.isnull(data):
        return "0", "0", "0", "micrometer"
    
    x_scale = "-"
    y_scale = "-"
    z_scale = "-"
    x_unit  = "-"
    y_unit  = "-"
    z_unit  = "-"
    
    match = re.findall(r'([XYZ]+): ([\d\.]+) ([\w\.]+)', data)
    for i in range(len(match)):
        for s in list(match[i][0]):
            if s == "X":
                x_scale = match[i][1]
                x_unit  = match[i][2]
            elif s == "Y":
                y_scale = match[i][1]
                y_unit  = match[i][2]
            elif s == "Z":
                z_scale = match[i][1]
                z_unit  = match[i][2]

    match = re.findall(r'([XYZ]+): NA', data)
    for i in range(len(match)):
        for s in list(match[i][0]):
            if s == "X":
                x_scale = 0
                if y_unit != "-":
                    x_unit = y_unit
                elif z_scale != "-":
                    x_unit = z_unit
            elif s == "Y":
                y_scale = 0
                if x_unit != "-":
                    y_unit = x_unit
                elif z_scale != "-":
                    y_unit = z_unit
            elif s == "Z":
                z_scale = 0
                if x_unit != "-":
                    z_unit = x_unit
                elif y_scale != "-":
                    z_unit = y_unit

    # ERROR: need to use the same unit among X,Y,Z-axes as precondition
    if x_unit != y_unit or y_unit != z_unit:
        x_unit = "-"

    return x_scale, y_scale, z_scale, x_unit

# Decoding scale and unit on T-axis
def searchT (data):
    if pd.isnull(data):
        return "0", "second"

    match = re.search(r"([\d\.]+) ([\w\.]+)", data)
    if match:
        if match.group(2) == "days":
            return match.group(1), "day"
        else:
            return match.group(1), match.group(2)
    else:
        return "-", "-"

# Decoding GO info.
def searchOntology (data):
    nlist = []
    olist = []
    
    if pd.isnull(data):
        return "-", "-"

    for d in data.split(','):
        match1 = re.search(r"^([\w\d\s\.]+) \(([\w\d]+):([\w\d]+)\)", d.strip())
        match2 = re.search(r"^([^\(\)]+)", d.strip())
        
        if match1 and match1.group(1) != 'NA':
            nlist.append(match1.group(1))
            olist.append(match1.group(2) + "_" + match1.group(3))
        elif match2 and match2.group(1) != 'NA':
            nlist.append(match2.group(1))
            olist.append("-")
            
    if len(nlist) >= 1:
        return ', '.join(nlist), ', '.join(olist)
    else:
        return "-", "-"

# Decoding PubMed info.
def searchPubMedID (data):
    if pd.isnull(data):
        return "-"    
    match = re.search(r"^https://www.ncbi.nlm.nih.gov/pubmed/(\d+)$", data)
    if match:
        return match.group(1)
    return "-"

# Reading all items of "common" sheet in Metadata_template.xlsx
def readTemplate (template_file):
    all_labels = []
    if not os.path.isfile(template_file):
        print ("ERROR: Template file not found. " + template_file)
    try:
        template = pd.read_excel(template_file, sheet_name='common')
        all_labels = list(template['Item'])
    except:
        print ("Cannot access:", template_file)
        
    return all_labels

# Checking consistency of all items of "common" sheet between Metadata_template.xlsx and Metadata.xlsx for one project
def checkFormat (input_file, template_file):    
    all_labels = readTemplate(template_file)
    if not all_labels:
        print ("Error of template excel")
        return -1
    
    try:
        data1 = pd.read_excel(input_file, sheet_name='common')
    except:
        print ("Cannot access common sheet:", input_file)
        return -1
    
    # Checking all items
    if list(data1['Item']) != all_labels:
        for label in list(data1['Item']):
            if not label in all_labels:
                print ("Error in common sheet", label)
                return -1
                    
    # Checking PubMed URL
    url = data1[data1['Item'].isin(['PubMed URL'])].iloc[0,1]
    if url != 'NA' and not pd.isnull(url):
        match = re.findall(r"https://www.ncbi.nlm.nih.gov/pubmed/(\d+)$", url)
        if len(match) != 1:
            print ("Error of PubMed URL in common sheet")
            return -1
    
    # Selecting items, in which all datasets have nothing in common
    tmp = data1[data1['Description'].isin(['Different by every dataset (refer to dataset sheet)'])]
    diff_count = len(tmp.index)
    diff_labels = []
    if diff_count != 0:
        diff_labels = list(tmp['Item'])
        
    # Cheching existence of the selected items in "dataset" sheet, excepting for "Note" item
    try:
        data2 = pd.read_excel(input_file, sheet_name='dataset')
        for label in diff_labels:
            if not label in list(data2.columns):
                print ("Error of %s in dataset sheet" % label)
                return -1
    except:
        print ("Cannot access dataset sheet:", input_file)
        return -1
    
    return 0

# Collecting and sorting metadata info for each dataset 
def decodeDataset(data2, dataset, dic1, all_labels):
    dic2 = {}
    dic2['dataset-name']  = "-"
    dic2['xScale']  = "-"
    dic2['yScale']  = "-"
    dic2['zScale']  = "-"
    dic2['xyzUnit'] = "-"
    dic2['tScale']  = "-"
    dic2['tUnit']   = "-"
    dic2['PMID']    = "-"
    dic2['organism-name']          = "-"
    dic2['organism_ontology'] = "-"
    dic2['biological-process']          = "-"
    dic2['biological-process_ontology'] = "-"
    dic2['cellular-component']          = "-"
    dic2['cellular-component_ontology'] = "-"
    
    # Copying "common" sheet
    for label in all_labels:
        dic2[label] = "-"
        if dic1[label] != None and dic1[label] != "-":
            dic2[label] = dic1[label]
        elif label in data2.columns and not pd.isnull(dataset[label]) and dataset[label] != "NA":
            dic2[label] = dataset[label]
    
    # dataset-name の処理
    dic2['dataset-name'] = dic2['LocalID']
    
    # Pattern matching
    source_default    = 'http://ssbd.qbic.riken.jp/data/(Project name)/source/(LocalID).zip'
    bdml_default      = 'http://ssbd.qbic.riken.jp/data/(Project name)/bdml/(LocalID)_bdml3.0.zip'
    omicsbdml_default = 'http://ssbd.qbic.riken.jp/data/(Project name)/bdml/(LocalID)_omicsbdml1.0.zip'
    pdpml_default     = 'http://ssbd.qbic.riken.jp/data/pdpml/(Project name)_pdpml1.0.xml'
    for label in all_labels:
        if label == 'Description' and dic2[label] == "-":
            if dic2['Kind'] == "Image data":
                dic2['Description'] = dic2['Title']
            elif dic2['Kind'] == "Quantitative data":
                match = re.search(r"BDML file for (.+)$", dic2['Title'])
                if match:
                    dic2['Description'] = match.group(1)
                else:
                    dic2['Description'] = dic2['Title']
                
        elif label == 'Source' and dic2[label] == source_default:
            if dic2['Kind'] == "Image data" :
                dic2[label] = 'http://ssbd.qbic.riken.jp/data/%s/source/%s.zip' % (dic2['Project name'], dic2['LocalID'])
                
            # For quantitative data, searching the corresponding to a set of images
            elif dic2['Kind'] == "Quantitative data":
                if len(data2.index) > 1:
                    b = []
                    b = data2[(data2['Kind'] == "Image data") & (data2['LocalID'] == dic2['LocalID'])].index # 組になる画像を探す
                    if len(b) == 1: # Exist
                        dic2[label] = 'http://ssbd.qbic.riken.jp/data/%s/source/%s.zip' % (dic2['Project name'], dic2['LocalID'])    
                    elif len(b) == 0: # Not exist
                        dic2[label] = "-"
                    else:
                        dic2[label] = "error1"
                else:
                    dic2[label] = "error2"

        elif label == 'BDML' and (dic2[label] == bdml_default or dic2[label] == omicsbdml_default):
            if dic2['Kind'] == "Quantitative data":
                if dic2[label] == bdml_default:
                    dic2[label] = 'http://ssbd.qbic.riken.jp/data/%s/bdml/%s_bdml3.0.zip' % (dic2['Project name'], dic2['LocalID'])
                elif dic2[label] == omicsbdml_default:
                    dic2[label] = 'http://ssbd.qbic.riken.jp/data/%s/bdml/%s_omicsbdml1.0.zip' % (dic2['Project name'], dic2['LocalID'])

            # For image data, searching the corresponding to a quantitative data
            elif dic2['Kind'] == "Image data":
                if len(data2.index) > 1:
                    b = []
                    b = data2[(data2['Kind'] == "Quantitative data") & (data2['LocalID'] == dic2['LocalID'])].index
                    if len(b) == 1: # Exist
                        if dic2[label] == bdml_default:
                            dic2[label] = 'http://ssbd.qbic.riken.jp/data/%s/bdml/%s_bdml3.0.zip' % (dic2['Project name'], dic2['LocalID'])
                        elif dic2[label] == omicsbdml_default:
                            dic2[label] = 'http://ssbd.qbic.riken.jp/data/%s/bdml/%s_omicsbdml1.0.zip' % (dic2['Project name'], dic2['LocalID'])
                    elif len(b) == 0: # Not exist
                        dic2[label] = "-"
                    else:
                        dic2[label] = "error1"
                else:
                    dic2[label] = "error2"
                    
        elif label == 'PDPML' and dic2[label] == pdpml_default:
            dic2[label] = 'http://ssbd.qbic.riken.jp/data/pdpml/%s_pdpml1.0.xml' % (dic2['Project name'])  
        elif label == 'XYZ-scale':
            dic2['xScale'], dic2['yScale'], dic2['zScale'], dic2['xyzUnit'] = searchXYZ(dic2[label])
        elif label == 'T-scale':
            dic2['tScale'], dic2['tUnit'] = searchT(dic2[label])
        elif label == 'PubMed URL':
            dic2['PMID'] = searchPubMedID(dic2[label])
        elif label == 'Organism':
            d1, d2 = searchOntology(dic2[label])
            dic2['organism'] = d1
            dic2['organism_ontology'] = d2
        elif label == 'Biological Process':
            d1, d2 = searchOntology(dic2[label])
            if d1 != '-' and d2 != '-':
                dic2['biological-process'] = d1
                dic2['biological-process_ontology'] = d2
        elif label == 'Cellular Component':
            d1, d2 = searchOntology(dic2[label])
            if d1 != '-' and d2 != '-':
                dic2['cellular-component'] = d1
                dic2['cellular-component_ontology'] = d2

    return dic2

def decodeMetadata(input_file, template_file, output_labels, target_kind, writer1, writer2):
    all_labels = readTemplate(template_file)
    
    if not os.path.isfile(input_file):
        print ("ERROR: Input file not found. " + input_file)
        return -1

    try:
        data1 = pd.read_excel(input_file, sheet_name='common')
    except:
        print ("Cannot access common sheet:", input_file)
        return -1
    
    # Creating dictionary on common sheet
    dic1 = {}
    diff_count = 0
    for label in all_labels:
        dic1[label]  = data1[data1['Item'].isin([label])].iloc[0,1]
        if dic1[label]  == 'Different by every dataset (refer to dataset sheet)':
            dic1[label] = None
            diff_count += 1
        elif pd.isnull(dic1[label]):
            dic1[label]  = "-"
             
    # Creating dataset info. if there is no derscription of "Different ..." in "common" sheet
    if diff_count == 0:
        data2 = pd.DataFrame({'Kind': [dic1['Kind']], 'LocalID': [dic1['LocalID']] })
    # ある場合は dataset シートを読む
    else:
        try:
            data2 = pd.read_excel(input_file, sheet_name='dataset')        
        except:
            print ("Cannot access dataset sheet:", input_file)
            return -1

    # Decoding dataset info. in "dataset" sheet row by row
    dataset_list = []
    for index, dataset in data2.iterrows():
        if dic1['Kind'] == target_kind or (dic1['Kind'] == None and dataset['Kind'] == target_kind):
            dic2 = decodeDataset(data2, dataset, dic1, all_labels)
            #print (dic2['dataset-name'])
            
            csv_list = []
            for label in output_labels:
                dic2[label] = dic2[label]
                csv_list.append(dic2[label])

            if len(csv_list) >= 1:
                writer1.writerow(csv_list)
                writer2.writerow(csv_list)
                dataset_list.append(dic2['dataset-name'])

    return len(dataset_list), dataset_list


In [224]:
# Generating CSV and SH files to import images and their metadata into SSBD-OMERO server
def readImageDataCSV(input_dir, output_dir, ssbd_dir, project_name):
    input_p_dir    = input_dir + project_name + '/'
    input_p_file   = output_dir + project_name + '/' + project_name + '.csv'

    output_p_dir   = output_dir + project_name + '/'
    output_p_file1 = output_p_dir + project_name + '_image_metadata.csv' # CSV files to import metadata of images
    output_p_file2 = output_p_dir + project_name + '_image.csv' # CSV files to import images
    output_p_file3 = output_p_dir + 'importImage.sh' # Script to import images
    output_p_file4 = output_p_dir + 'zipImage.sh' # Script to zip images

    ssbd_p_dir     = ssbd_dir + project_name + '/source/'

    if not os.path.isfile(input_p_file):
        print ("ERROR: Input CSV file not found:" + input_p_file)
        return project_name, 0, 0, {}
    
    df1 = pd.read_csv(input_p_file)
    
    df2 = df1[df1['kind'] == "Image data"]

    if not df2.empty and df2.shape[0] >= 1:
        # Checking for the existence of "source" directory
        if not os.path.isdir(input_p_dir):
            print ("ERROR: Input dir not found:" + input_p_dir)
            return project_name, 0, 0, {}
        
    # Creating files.
    fp1 = csv.writer(open(output_p_file1, "w"), lineterminator='\n')
    fp2 = csv.writer(open(output_p_file2, "w"), lineterminator='\n')
    fp3 = open(output_p_file4, "w")
    fp3.writelines("#!/bin/sh\n")

    org_dic = {} #the number of the datasets for each kind of the organism
    total_image_count = 0 # the number of image files for each project

    for i in df2.index:
        dataset_name = df2.iloc[i]["localID"]
        input_d_dir = input_p_dir + 'source/' + dataset_name + '/'
        if not os.path.isdir(input_d_dir):
            print ("Not exist", input_d_dir)
        else:
            image_count = 0
            ext_dic   = {}
            for file_name in os.listdir(input_d_dir):
                if os.path.isfile(input_d_dir + file_name) and file_name != '.DS_Store':
                    ext = os.path.splitext( file_name )[1]
                    if ext in ext_dic.keys():
                        ext_dic[ext] += 1
                    else:
                        ext_dic[ext] = 1
                    image_count += 1
                    
            if image_count > 0:
                total_image_count += image_count
            
            # Generating CSV file (TODO: commonalize CSV files)
            dic = df2.iloc[i].to_dict()
            csvlist1 = [dic['project-name'], dic['dataset-name'], dic['dataset-name'], dic['summary'], dic['license'], dic['contact-name'], \
                        dic['organization'], dic['department'], dic['laboratory'], dic['contributors'], dic['title'], dic['organism'], \
                        dic['PMID'] , dic['x-scale'], dic['y-scale'], dic['z-scale'], dic['xyz-unit'], dic['t-scale'], dic['t-unit'], \
                        dic['source'], dic['bdml'], dic['exlinks']]
            fp1.writerow(csvlist1)
            #print (csvlist1)

            # Summarzing the number of organism
            if dic['organism'] in org_dic:
                org_dic[dic['organism']] += 1
            else:
                org_dic[dic['organism']] = 1

            # Generating import commands (project_name, dataset_name, file extension, #images)
            ssbd_d_dir = ssbd_p_dir + dataset_name + "/"
            if len(ext_dic.keys()) != 1:
                print ("Extension ERROR in Project", project_name, "Dataset", dataset_name, ":", ext_dic.keys())
            csvlist2 = [dic['project-name'], dic['dataset-name'], ext, ssbd_d_dir, str(image_count)]
            fp2.writerow(csvlist2)
        
            # Generating zip commands
            cmd = "cd source; rm %s.zip; zip %s.zip -r %s\n" % (dataset_name, dataset_name, dataset_name)
            fp3.writelines(cmd)

    fp3.close()
    
    # Generating script to import images for each project
    if len(df2.index) > 0:
        fp4 = open(output_p_file3, 'w')
        fp4.writelines("#!/bin/sh\n")
        cmd = 'python /OMERO/importImages.py %s_image.csv' % (project_name)
        fp4.writelines(cmd)
        fp4.close()
        
    else:
        if os.path.isfile(output_p_file1):
            os.remove(output_p_file1)
        if os.path.isfile(output_p_file2):
            os.remove(output_p_file2)
        if os.path.isfile(output_p_file3):
            os.remove(output_p_file3)
        if os.path.isfile(output_p_file4):
            os.remove(output_p_file4)
        
    return len(df2.index), total_image_count, org_dic

def getProjects (input_dir, project_no):
    pattern = re.compile(r'\d+')
    datasets = []
    if len(project_no) != 0:
        path_dict = {}
        file_dict = {}
        for project_name in sorted(os.listdir(input_dir)):
            full_path = input_dir + project_name + "/"
            if os.path.isdir(full_path): 
                match_obj = pattern.match(project_name) 
                if match_obj and int(match_obj.group(0)) in project_no:
                    datasets.append(project_name)
    return datasets

## Step 1: Check for existance of the EXCEL file described metadata per project

Precondition: Project name is equal to EXCEL-file name.


In [225]:
print ("INPUT_DIR:", INPUT_DIR)

ok_projects = []
tmp_projects = []

pattern = re.compile(r'\d+')

dir_dict = {}
file_dict = {}
for project_name in sorted(os.listdir(INPUT_DIR)):
    input_p_dir = INPUT_DIR + project_name + "/"
    input_p_file = INPUT_DIR + project_name + "/" + project_name + ".xlsx"
    
    if os.path.isdir(input_p_dir):
        match_obj = pattern.match(project_name)
        if match_obj:
            if int(match_obj.group(0)) in TARGET_PROJECTS:         
                if os.path.isfile(input_p_file): 
                    print ("OK:", project_name)
                    ok_projects.append(int(match_obj.group(0)))
                    dir_dict[project_name] = input_p_dir
                    file_dict[project_name] = input_p_file
                else: 
                    print ("Excel-file name error:", os.path.basename(input_p_file))
            else:
                tmp_projects.append(int(match_obj.group(0)))
        else:
            print ("Directory name error:", project_name)

print ("Target:", sorted(ok_projects))
print ("Not target:", sorted(tmp_projects))


INPUT_DIR: Collection_test/
OK: 20-Azuma-WormMembrane
Directory name error: metadata
Target: [20]
Not target: []


## Step 2: Check format of the EXCEL file based on Metadata_template.xlsx

In [226]:
# Mappping item names of EXCEL file to CSV file
# Item name for EXCEL
output_labels1 = ['Project name', 'dataset-name', 'LocalID', 'Contact name', 'E-mail', 'Organization', 'Department', \
                 'Laboratory', 'Address', 'License', 'Contributors', 'Method summary', 'PMID', \
                 'Kind', 'Title', 'Description', 'organism', 'organism_ontology', 'Datatype', 'Basedon', \
                 'xScale', 'yScale', 'zScale', 'xyzUnit', 'tScale', 'tUnit', 'dblink', 'External links', \
                 'Source', 'BDML', 'PDPML', \
                 'biological-process', 'biological-process_ontology', 'cellular-component', 'cellular-component_ontology']

# Item name for CSV
output_labels2 = ['project-name', 'dataset-name', 'localID', 'contact-name', 'E-mail', 'organization', 'department', \
                 'laboratory', 'address', 'license', 'contributors', 'summary', 'PMID', \
                 'kind', 'title', 'description', 'organism', 'organism_ontology', 'datatype', 'basedon', \
                 'x-scale', 'y-scale', 'z-scale', 'xyz-unit', 't-scale', 't-unit', 'dblink', 'exlinks', \
                 'source', 'bdml', 'pdpml', \
                 'biological-process', 'biological-process_ontology', 'cellular-component', 'cellular-component_ontology']

print ("TEMPLATE_FILE:", TEMPLATE_FILE)
term_list = readTemplate(TEMPLATE_FILE)

if len(output_labels1) != len(output_labels2):
    print ("Error: cannot correspond output_labels1 and output_label2")

print ("Available terms in CSV:")
for i in range(len(output_labels1)):
    if not output_labels1[i] in term_list:
        print ("\t", output_labels2[i])
    else:
        print ("\t", output_labels2[i], "(xlsx:", output_labels1[i], ")")

error = 0
count = 0
for project_name, input_p_file in file_dict.items():
    result = checkFormat(input_p_file, TEMPLATE_FILE) # Metadata_template と同じ形式かをチェック
    if result == 0:
        print ("OK", input_p_file)
    else:
        print ("NG", input_p_file)
        error += result
    count += 1       

print ("#project = %d" % count)
print ("#error = %d" % error)

TEMPLATE_FILE: Collection_test/Metadata_template.xlsx
Available terms in CSV:
	 project-name (xlsx: Project name )
	 dataset-name
	 localID (xlsx: LocalID )
	 contact-name (xlsx: Contact name )
	 E-mail (xlsx: E-mail )
	 organization (xlsx: Organization )
	 department (xlsx: Department )
	 laboratory (xlsx: Laboratory )
	 address (xlsx: Address )
	 license (xlsx: License )
	 contributors (xlsx: Contributors )
	 summary (xlsx: Method summary )
	 PMID
	 kind (xlsx: Kind )
	 title (xlsx: Title )
	 description (xlsx: Description )
	 organism
	 organism_ontology
	 datatype (xlsx: Datatype )
	 basedon (xlsx: Basedon )
	 x-scale
	 y-scale
	 z-scale
	 xyz-unit
	 t-scale
	 t-unit
	 dblink (xlsx: dblink )
	 exlinks (xlsx: External links )
	 source (xlsx: Source )
	 bdml (xlsx: BDML )
	 pdpml (xlsx: PDPML )
	 biological-process
	 biological-process_ontology
	 cellular-component
	 cellular-component_ontology
OK Collection_test/20-Azuma-WormMembrane/20-Azuma-WormMembrane.xlsx
#project = 1
#error = 

## Step 3: Generating all CSV and SH files


In [219]:
os.makedirs(OUTPUT_DIR, exist_ok=True) #  # Creating working directory if not exist

print ("Output directory was created:", OUTPUT_DIR)
print ("Targets:", TARGET_PROJECTS)
    
output_file1 = OUTPUT_DIR + 'metadata-all.csv'
output_file2 = OUTPUT_DIR + 'copyData.sh'
output_file5 = OUTPUT_DIR + 'addAnnotations.sh'
output_file6 = OUTPUT_DIR + 'makeMD5.sh'

fp1 = open(output_file1, "w") # Creating CSV file [metadata-all.csv] including metadata for all projects 
writer1 = csv.writer(fp1, lineterminator='\n')
writer1.writerow(output_labels2)

fp2 = open(output_file2, "w") # Creating SH script [copyData.sh] to copy metadata into SSBD-OMERO server
fp2.write('#!/bin/sh\n')
fp2.write('scp -r %s* %s@%s:%s.\n' % (OUTPUT_DIR, SERVER_USER, SERVER_HOST, SERVER_DIR))

# Processing for each project
for project_name, full_file in file_dict.items():
    input_file   = INPUT_DIR + project_name + '/' + project_name + '.xlsx'
    input_p_dir  = INPUT_DIR + project_name + '/'
        
    output_p_dir = OUTPUT_DIR + project_name + '/'
    output_file3 = output_p_dir + project_name + '.xlsx'
    output_file4 = output_p_dir + project_name + '.csv' 
        
    os.makedirs(output_p_dir, exist_ok=True) # Creating project directory if not exist
        
    print(project_name)
        
    # Creating CSV file including metadata for one target project
    fp4 = open(output_file4, "w")
    writer2 = csv.writer(fp4, lineterminator='\n') #, quoting=csv.QUOTE_NONE
    writer2.writerow(output_labels2) # 項目名の出力
                
    # Outputing information of images data and quantitative data per project
    for kind in ["Image data", "Quantitative data"]:
        (n, dataset_list) = decodeMetadata(full_file, TEMPLATE_FILE, output_labels1, kind, writer1, writer2)
        print ("\t%s: #%d" % (kind, n))
        if n > 0:
            for dataset_name in dataset_list:
                input_d_dir = ""
                if kind == "Image data":
                    input_d_dir = input_p_dir + 'source/' + dataset_name + '/'
                elif kind == "Quantitative data":
                    input_d_dir = input_p_dir + 'bdml/' + dataset_name + '/'

                if not os.path.isdir(input_d_dir):
                    print ("\t\tNot exist", input_d_dir)
        
    fp4.close()

    # Adding scp command into script [copyData.sh] to copy source folders into SSBD-OMERO server
    dir2 = '%s@%s:%s%s/.' % (SERVER_USER, SERVER_HOST, SERVER_DIR, project_name)
    dir1 = input_p_dir + 'data/'
    if os.path.isdir(dir1):
        print ("\tneeded scp:", dir1)
        fp2.write("scp -r %s %s\n" % (dir1, dir2))
    dir1 = input_p_dir + 'source/'
    if os.path.isdir(dir1):
        print ("\tneeded scp:", dir1)
        fp2.write("scp -r %s %s\n" % (dir1, dir2))
    
    shutil.copy(input_file, output_file3) # Copying EXCEL files
    
fp1.close()
fp2.close()


Output directory was created: Collection_test/metadata/
Targets: [20, 115]
20-Azuma-WormMembrane
	Image data: #1
	Quantitative data: #1
	needed scp: Collection_test/20-Azuma-WormMembrane/source/


In [220]:
fp5 = open(output_file5, 'w')
fp6 = open(output_file6, 'w')

fp5.writelines('#!/bin/sh\n\n')
fp6.writelines('#!/bin/zsh\n\ndirs=(\\\n')

total_imageset_count = 0
total_image_count = 0
project_list = getProjects(OUTPUT_DIR, TARGET_PROJECTS)
for project_name in project_list:
    imageset_count, image_count, org_dic  = readImageDataCSV(INPUT_DIR, OUTPUT_DIR, SERVER_DIR, project_name)
    print (project_name, ", #dataset:", imageset_count, ", #image:", image_count, org_dic)
    
    # Generating script to annotate metadata of images on OMERO
    if imageset_count != 0:
        cmd1 = "python /OMERO/addAnnotations.py " + SERVER_DIR + project_name + "/" + project_name + "_image_metadata.csv" + "\n"
        cmd2 = "\t\"" + SERVER_DIR + project_name + "/source/\"\\\n"
        fp5.writelines(cmd1)
        fp6.writelines(cmd2)

    total_imageset_count += imageset_count
    total_image_count += image_count

    fp6.writelines('\t)\n\nfor dir in ${dirs}; do\n\techo ${dir}\n\tcd ${dir}\n\tfor file in `find . -name "*.zip" | sort`;\n\tdo\n\t\tmd5_file="${file}.md5"\n\t\t/usr/bin/md5sum $file > $md5_file\n\t\techo $md5_file\n\tdone;\ndone;\n')

fp5.close()
fp6.close()

print ("\nThe following CSV files and scripts were created:")
print (output_file1)
print (output_file2)
print (output_file5)
print (output_file6)

print ("\nTotal #project:", len(project_list), ", Total #dataset of images:", total_imageset_count, ", Total #image:", total_image_count)
print ("Done.")

20-Azuma-WormMembrane , #dataset: 1 , #image: 1 {'C. elegans': 1}

The following CSV files and scripts were created:
Collection_test/metadata/metadata-all.csv
Collection_test/metadata/copyData.sh
Collection_test/metadata/addAnnotations.sh
Collection_test/metadata/makeMD5.sh

Total #project: 1 , Total #dataset of images: 1 , Total #image: 1
Done.
